In [3]:
import json
import pandas as pd
import numpy as np
import datetime

In [5]:
import os
cwd = os.getcwd()

In [17]:

### need to import the parent folder
covid_path = cwd + "\\data\\yelp_academic_dataset_covid_features.json"
business_path = cwd +"\\data\\yelp_academic_dataset_business.json"
user_path = cwd +"\\data\\yelp_academic_dataset_user.json"
review_path_new = cwd +"\\data\\yelp_academic_dataset_review.json"
review_path_old = cwd +"\\data\\yelp_academic_dataset_review.json"

In [24]:
rest_2021 = pd.read_csv("restaurant_2021.csv")
rest_covid = pd.read_csv("restaurant_covid.csv")

## Review data

In [85]:
review_data = []
with open(review_path, "r", encoding="utf-8") as f:
    Lines = f.readlines()
    for l in Lines:
        review_data.append(json.loads(l))

In [87]:
review = pd.DataFrame.from_dict(review_data)
review = review.drop(columns = ["review_id", "user_id"])

In [118]:
review['after_covid'] = 0
for i,t in enumerate(review.date):
    if datetime.datetime.strptime(t, '%Y-%m-%d %H:%M:%S').date() > datetime.date(2020,3,1):
        review['after_covid'].iloc[i] = 1

In [120]:
len(review.after_covid)

8635403

In [119]:
sum(review.after_covid)

491988

In [185]:
review.to_csv("review_raw.csv", index=False)

In [18]:
review_before_covid = review[review.after_covid == 0].sample(frac = 0.1)

In [123]:
review_after_covid = review[review.after_covid == 1]
review_after_covid.to_csv("review_after_covid.csv", index = False)

In [7]:
review = pd.read_csv("review_raw.csv")
review_after_covid = pd.read_csv("review_after_covid.csv")

In [16]:
merge_after_covid = review_after_covid.merge(rest_2021, left_on="business_id", right_on="business_id")
merge_after_covid.to_csv("merge_after_covid_2021.csv", index = False)

In [17]:
merge_after_covid.shape

(331839, 20)

In [22]:
merge_before_covid = review_before_covid.merge(rest_2021, left_on="business_id", right_on="business_id")
merge_before_covid.to_csv("merge_before_covid_2021.csv", index = False)

In [21]:
merge_before_covid.shape

(572395, 20)

# highlights with the covid + restaurant 2020 data NEED REVIEW 2020

In [27]:
covid_review = pd.read_csv("review_restaurant_covid.csv") #this is half of the joined data

C:\Users\hefzh\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [116]:
covid_review.iloc[2]

business_id                                                     nlxHRv1zXGT0c0K51q3jDg
user_rating                                                                        5.0
useful                                                                               2
funny                                                                                0
cool                                                                                 0
text                                 This is definitely my favorite fast food sub s...
city                                                                         Henderson
state                                                                               NV
postal_code                                                                      89014
latitude                                                                     36.059969
longitude                                                                  -115.036032
business_rating                            

In [29]:
highlights_cat = covid_review.highlights.unique()

In [44]:
def flatten(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [80]:
covid_review.highlights = covid_review.highlights.replace(np.nan, "NA")
l = []
for i in highlights_cat:
    if i is not np.nan:
        l.append(i.split(", "))
l_flat = flatten(l)
l_flat_unique = []
for i in set(l_flat):
    if "during_covid_19" in i:
        l_flat_unique.append(i)
for i, h in enumerate(covid_review.highlights):
    for col in l_flat_unique:
        covid_review[col].iloc[i] = 1 if col in h else 0 

KeyboardInterrupt: 